In [1]:
import pandas as pd
import numpy as np

In [2]:
#load train and test data

In [2]:
#train_data = pd.read_excel('train.xlsm',header=0)
train_data = pd.read_csv('train.csv',nrows=500000)

In [4]:
#test_data = pd.read_excel('test.xlsm',header=0)
test_data = pd.read_csv('test.csv' )

In [5]:
test_data.head(5)

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
#check for missing values

In [7]:
has_missing_values = train_data.isnull().any().any()
print("Are there any missing values?", has_missing_values)

Are there any missing values? True


In [8]:
missing_count = train_data.isnull().sum()
print("Missing values per column:\n", missing_count)

Missing values per column:
 key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    5
dropoff_latitude     5
passenger_count      0
dtype: int64


In [9]:
# Removing rows with any missing values
train_data = train_data.dropna()

In [10]:
missing_count_2 = train_data.isnull().sum()
print("Missing values per column:\n", missing_count_2)

Missing values per column:
 key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Convert 'pickup_datetime' from string to datetime
train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'])

# Extract features from datetime
train_data['year'] = train_data['pickup_datetime'].dt.year
train_data['month'] = train_data['pickup_datetime'].dt.month
train_data['day'] = train_data['pickup_datetime'].dt.day
train_data['hour'] = train_data['pickup_datetime'].dt.hour
train_data['weekday'] = train_data['pickup_datetime'].dt.weekday

# Prepare features and target variable
X = train_data[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
                       'dropoff_latitude', 'passenger_count', 'year', 'month', 'day', 
                       'hour', 'weekday']]
y = train_data['fare_amount']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

rmse


9.90778562295318

In [12]:
train_data.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17,0
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0,3
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4,5
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7,1


In [13]:
# Load the test data
# Convert 'pickup_datetime' from string to datetime
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'])

# Extract features from datetime
test_data['year'] = test_data['pickup_datetime'].dt.year
test_data['month'] = test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.day
test_data['hour'] = test_data['pickup_datetime'].dt.hour
test_data['weekday'] = test_data['pickup_datetime'].dt.weekday

# Prepare features for the test set
X_test = test_data[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
                    'dropoff_latitude', 'passenger_count', 'year', 'month', 'day', 
                    'hour', 'weekday']]

# Predict on the test set
y_test_pred = model.predict(X_test)

# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({
    'key': test_data['key'],
    'fare_amount': y_test_pred
})

predictions_df.head()


,key,fare_amount
0,2015-01-27 13:08:24.0000002,12.859820
1,2015-01-27 13:08:24.0000003,12.859875
2,2011-10-08 11:53:44.0000002,11.263131
3,2012-12-01 21:12:12.0000002,11.790264
4,2012-12-01 21:12:12.0000003,11.790268


In [ ]:
#calculate the geographical distance using the Haversine formula to see how this affects our model.

In [14]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r

# Apply the Haversine function to calculate distances for both train and test data
train_data['distance'] = train_data.apply(
    lambda row: haversine(row['pickup_longitude'], row['pickup_latitude'],
                          row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
                       'dropoff_latitude', 'passenger_count', 'year', 'month', 'day', 
                       'hour', 'weekday', 'distance']],
   train_data['fare_amount'], test_size=0.2, random_state=0)

# Retrain the model with the new feature
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse


9.90616195097457

In [21]:
train_data.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17,0,1.030764
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16,1,8.450134
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0,3,1.389525
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4,5,2.799270
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7,1,1.999157


In [15]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100,random_state = 42, n_jobs = -1, max_depth = 10)
#model.fit(X_train, y_train)

In [16]:
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42)

In [17]:
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

5.05261916047918

In [14]:
lm_sklearn = model.fit(X_train, y_train) # Fit/"train"/"learn" a linear regression with vector Y as dependent and matrix X as independent

print("Intercept = ",lm_sklearn.intercept_) # Print the resultant/estimated/"learned" model intercept

print("Model coefficients = ", lm_sklearn.coef_) # Print the resultant/estimated/"learned" model coefficients (in order of variables in X)

print("R^2 =",lm_sklearn.score(X_train,y_train)) 

Intercept =  -1256.494678364014
Model coefficients =  [ 2.06007165e-03 -6.14903729e-05  2.64020812e-03 -1.23323426e-03
  9.16658176e-02  6.30134588e-01  1.09108051e-01  3.79510970e-03
 -2.58955429e-02  3.89621739e-03  2.22280268e-01]
R^2 = 0.015891553293800875


In [ ]:
#prepare the test data for model prediction

In [18]:
# Extract features from datetime for the test data
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'])
test_data['year'] = test_data['pickup_datetime'].dt.year
test_data['month'] = test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.day
test_data['hour'] = test_data['pickup_datetime'].dt.hour
test_data['weekday'] = test_data['pickup_datetime'].dt.weekday

In [19]:
# Prepare the test set with polynomial features for distance
test_data['distance'] = test_data.apply(
    lambda row: haversine(row['pickup_longitude'], row['pickup_latitude'],
                          row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
#test_data['scaled_distance'] = scaler.transform(test_data[['distance']])
#X_test_poly = poly_transformer.transform(test_data[['scaled_distance']])

In [20]:
test_data.head(5)

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday,distance
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13,1,2.323260
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13,1,2.425353
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11,5,0.618628
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21,5,1.961033
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21,5,5.387301


In [24]:
# Prepare the full set of features including original and polynomial for the test set
X_test_prepared = test_data[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
                       'dropoff_latitude', 'passenger_count', 'year', 'month', 'day', 
                       'hour', 'weekday', 'distance']]


# Predict on the fully prepared test set with polynomial features
y_test_pred = model.predict(X_test_prepared)

# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({
    'key': test_data['key'],
    'fare_amount': y_test_pred
})

predictions_df.head()



,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.551118
1,2015-01-27 13:08:24.0000003,10.802165
2,2011-10-08 11:53:44.0000002,5.009345
3,2012-12-01 21:12:12.0000002,7.960535
4,2012-12-01 21:12:12.0000003,15.862277


In [25]:
test_data['fare_amount'] = y_test_pred

In [26]:
submission = pd.DataFrame(
    {'key': test_data['key'], 'fare_amount': test_data['fare_amount']},
    columns = ['key', 'fare_amount'])

# save the predictions to a CSV file
submission.to_csv('submissions1.csv', index = False)